In [0]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

# New Section

In [1]:
!mkdir -p drive
!google-drive-ocamlfuse drive

fuse: mountpoint is not empty
fuse: if you are sure this is safe, use the 'nonempty' mount option


In [0]:
!pip install -q keras

In [3]:
import time
start_time = time.time()
from sklearn.model_selection import train_test_split
import sys, os, re, csv, codecs, numpy as np, pandas as pd
np.random.seed(32)
#os.environ["OMP_NUM_THREADS"] = "4"
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import text, sequence
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, Conv1D, CuDNNLSTM, CuDNNGRU
from keras.layers import Bidirectional, GlobalMaxPool1D, MaxPooling1D, Add, Flatten, TimeDistributed
from keras.layers import GlobalAveragePooling1D, GlobalMaxPooling1D, concatenate, SpatialDropout1D
from keras.models import Model, load_model
from keras import initializers, regularizers, constraints, optimizers, layers, callbacks
from keras import backend as K
from keras.engine import InputSpec, Layer
from keras.callbacks import *
from keras.optimizers import Adam, RMSprop
from keras.callbacks import EarlyStopping, ModelCheckpoint, LearningRateScheduler
from keras.layers import GRU, BatchNormalization, Conv1D, MaxPooling1D
import pickle
print('imports complete')

Using TensorFlow backend.


imports complete


In [4]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [0]:
import logging
from sklearn.metrics import roc_auc_score
from keras.callbacks import Callback

class RocAucEvaluation(Callback):
    def __init__(self, validation_data=(), interval=1):
        super(Callback, self).__init__()

        self.interval = interval
        self.X_val, self.y_val = validation_data

    def on_epoch_end(self, epoch, logs={}):
        if epoch % self.interval == 0:
            y_pred = self.model.predict(self.X_val, verbose=0)
            score = roc_auc_score(self.y_val, y_pred)
            print("\n ROC-AUC - epoch: {:d} - score: {:.6f}".format(epoch+1, score))

In [6]:
train = pd.read_csv("drive/jigsaw/train.csv")
test = pd.read_csv("drive/jigsaw/test.csv")
embedding_path = "drive/jigsaw/crawl-300d-2M.vec"
#embedding_path = "../embeddings/glove.840B.300d.txt"
embed_size = 300
max_features = 150000
max_len = 200
maxlen = 200
print('data read in')

data read in


In [7]:
# Construct tuples out of non-toxic column values
label_cols = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
train['mytuples'] = train[label_cols].apply(tuple, axis=1)
print('mytuples generated')

mytuples generated


In [8]:
# Construct a column for no toxic label
#train['none'] = 1-train[label_cols].max(axis=1)
train.describe()

X_train = train["comment_text"].fillna("fillna").values
y_train = train[label_cols].values
X_test = test["comment_text"].fillna("fillna").values
nlabels = y_train.shape[1]
print(train.describe())

print('test train ready after processing missing values')

               toxic   severe_toxic        obscene         threat  \
count  159571.000000  159571.000000  159571.000000  159571.000000   
mean        0.095844       0.009996       0.052948       0.002996   
std         0.294379       0.099477       0.223931       0.054650   
min         0.000000       0.000000       0.000000       0.000000   
25%         0.000000       0.000000       0.000000       0.000000   
50%         0.000000       0.000000       0.000000       0.000000   
75%         0.000000       0.000000       0.000000       0.000000   
max         1.000000       1.000000       1.000000       1.000000   

              insult  identity_hate  
count  159571.000000  159571.000000  
mean        0.049364       0.008805  
std         0.216627       0.093420  
min         0.000000       0.000000  
25%         0.000000       0.000000  
50%         0.000000       0.000000  
75%         0.000000       0.000000  
max         1.000000       1.000000  
test train ready after processing mi

In [9]:
print(' Cleaning ...')
# PREPROCESSING PART
repl = {
    "yay!": " good ",
    "yay": " good ",
    "yaay": " good ",
    "yaaay": " good ",
    "yaaaay": " good ",
    "yaaaaay": " good ",
    ":/": " bad ",
    ":&gt;": " sad ",
    ":')": " sad ",
    ":-(": " frown ",
    ":(": " frown ",
    ":s": " frown ",
    ":-s": " frown ",
    "&lt;3": " heart ",
    ":d": " smile ",
    ":p": " smile ",
    ":dd": " smile ",
    "8)": " smile ",
    ":-)": " smile ",
    ":)": " smile ",
    ";)": " smile ",
    "(-:": " smile ",
    "(:": " smile ",
    ":/": " worry ",
    ":&gt;": " angry ",
    ":')": " sad ",
    ":-(": " sad ",
    ":(": " sad ",
    ":s": " sad ",
    ":-s": " sad ",
    r"\br\b": "are",
    r"\bu\b": "you",
    r"\bhaha\b": "ha",
    r"\bhahaha\b": "ha",
    r"\bdon't\b": "do not",
    r"\bdoesn't\b": "does not",
    r"\bdidn't\b": "did not",
    r"\bhasn't\b": "has not",
    r"\bhaven't\b": "have not",
    r"\bhadn't\b": "had not",
    r"\bwon't\b": "will not",
    r"\bwouldn't\b": "would not",
    r"\bcan't\b": "can not",
    r"\bcannot\b": "can not",
    r"\bi'm\b": "i am",
    "m": "am",
    "r": "are",
    "u": "you",
    "haha": "ha",
    "hahaha": "ha",
    "don't": "do not",
    "doesn't": "does not",
    "didn't": "did not",
    "hasn't": "has not",
    "haven't": "have not",
    "hadn't": "had not",
    "won't": "will not",
    "wouldn't": "would not",
    "can't": "can not",
    "cannot": "can not",
    "i'm": "i am",
    "m": "am",
    "i'll" : "i will",
    "its" : "it is",
    "it's" : "it is",
    "'s" : " is",
    "that's" : "that is",
    "weren't" : "were not",
}

keys = [i for i in repl.keys()]

new_train_data = []
new_test_data = []
ltr = train["comment_text"].tolist()
lte = test["comment_text"].tolist()
for i in ltr:
    arr = str(i).split()
    xx = ""
    for j in arr:
        j = str(j).lower()
        if j[:4] == 'http' or j[:3] == 'www':
            continue
        if j in keys:
            # print("inn")
            j = repl[j]
        xx += j + " "
    new_train_data.append(xx)
for i in lte:
    arr = str(i).split()
    xx = ""
    for j in arr:
        j = str(j).lower()
        if j[:4] == 'http' or j[:3] == 'www':
            continue
        if j in keys:
            # print("inn")
            j = repl[j]
        xx += j + " "
    new_test_data.append(xx)
train["new_comment_text"] = new_train_data
test["new_comment_text"] = new_test_data

trate = train["new_comment_text"].tolist()
tete = test["new_comment_text"].tolist()
for i, c in enumerate(trate):
    trate[i] = re.sub('[^a-zA-Z ?!]+', '', str(trate[i]).lower())
for i, c in enumerate(tete):
    tete[i] = re.sub('[^a-zA-Z ?!]+', '', tete[i])
train["comment_text"] = trate
test["comment_text"] = tete
del trate, tete
train.drop(["new_comment_text"], axis=1, inplace=True)
test.drop(["new_comment_text"], axis=1, inplace=True)
print('cleaning complete')

 Cleaning ...
cleaning complete


In [0]:
# Lowercase all comments
train["lcase_comment_text"] = train["comment_text"].astype(str).str.lower()
test["lcase_comment_text"] = test["comment_text"].astype(str).str.lower()
train.drop('comment_text', axis=1, inplace=True)
test.drop('comment_text', axis=1, inplace=True)

In [0]:
# Construct a dataframe that reverses the comment_text
def reverse_string(row):
    return " ".join(row.split(" ")[::-1])

train['rcomment_text'] = train['lcase_comment_text'].apply(lambda x: reverse_string(x))
test['rcomment_text'] = test['lcase_comment_text'].apply(lambda x: reverse_string(x))

In [12]:
# Prepare numpy array to manage data
X_train = train["lcase_comment_text"].str.lower().values
rX_train = train["rcomment_text"].str.lower().values
y_train = train[["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]].values
X_test = test["lcase_comment_text"].values
rX_test = test["rcomment_text"].values

# Each row of X_train is one string, so the dimension of X_train is number of rows
print('shape of X_train = ',X_train.shape)
print('shape of rX_train = ',rX_train.shape)
print('shape of X_test = ',X_test.shape)
print('shape of rX_test = ',rX_test.shape)
print('shape of y_train = ',y_train.shape)

print('data transferred to numpy format')

shape of X_train =  (159571,)
shape of rX_train =  (159571,)
shape of X_test =  (153164,)
shape of rX_test =  (153164,)
shape of y_train =  (159571, 6)
data transferred to numpy format


In [13]:
new_X_train = np.concatenate((X_train, rX_train),axis=0)
new_y_train = np.vstack((y_train,y_train))
print('shape of new_X_train = ', new_X_train.shape)
print('shape of new_y_train = ',new_y_train.shape)

shape of new_X_train =  (319142,)
shape of new_y_train =  (319142, 6)


In [14]:
tok=text.Tokenizer(num_words=max_features,lower=True)
tok.fit_on_texts(list(new_X_train)+list(X_test)+list(rX_test))
X_train=tok.texts_to_sequences(new_X_train)
X_test=tok.texts_to_sequences(X_test)
rX_test=tok.texts_to_sequences(rX_test)
print('len of X_train = ',len(X_train))
print('len of X_test = ',len(X_test))
print('len of rX_test = ',len(rX_test))

x_train=sequence.pad_sequences(X_train,maxlen=maxlen)
x_test=sequence.pad_sequences(X_test,maxlen=maxlen)
rx_test=sequence.pad_sequences(rX_test,maxlen=maxlen)
print('len of x_train = ',len(x_train))
print('len of x_test = ',len(x_test))
print('len of rx_test = ',len(rx_test))
print('text tokenization and sequence padding complete')

len of X_train =  319142
len of X_test =  153164
len of rX_test =  153164
len of x_train =  319142
len of x_test =  153164
len of rx_test =  153164
text tokenization and sequence padding complete


In [0]:
# Split X_train, y_train into train and validation subsets
def split_train_data1(X_train, y_train, split_val_frac, seed_val=233):
    # Now construct an array of indices and shuffle them
    shuffled_idx = np.random.RandomState(seed=seed_val).permutation(len(y_train))
    valid_size = int(np.round(len(y_train)*split_val_frac))
    valid_indices = shuffled_idx[:valid_size]
    train_indices = shuffled_idx[valid_size:]
    
    # Save the train and valid indices
    pickle.dump( train_indices, open("drive/jigsaw/train_indices.p","wb"))
    pickle.dump( valid_indices, open("drive/jigsaw/valid_indices.p","wb"))
    
    # Return split dataset
    return X_train[train_indices], X_train[valid_indices,], y_train[train_indices], y_train[valid_indices,]

In [0]:
# Split X_train, y_train into train and validation subsets using preset
# selection of train and validation indices
def split_train_data2(X_train, y_train, split_val_frac, seed_val=233):    
    # Read train and validation indices
    train_indices = pickle.load( open("drive/jigsaw/train_indices.p","rb"))
    valid_indices = pickle.load( open("drive/jigsaw/valid_indices.p","rb"))
    
    # Return split dataset
    return X_train[train_indices], X_train[valid_indices,], y_train[train_indices], y_train[valid_indices,]

In [0]:
# Perform train-validation split
#X_tra, X_val, y_tra, y_val, weights_tra, weights_val = train_test_split(x_train, y_train, weight_array, test_size=0.10, random_state=233)
#print('train-validation split complete')

In [18]:
# Perform train-validation split
rndm_state=233
split_val_frac=0.1
X_tra, X_val, y_tra, y_val = split_train_data2(x_train, new_y_train, split_val_frac, rndm_state)
print('train-validation split complete')

train-validation split complete


In [19]:
# Print out the shape of all arrays with split data
print('shape of X_tra = ',X_tra.shape)
print('shape of X_val = ',X_val.shape)
print('y_tra.shape = ',y_tra.shape)
print('y_val.shape = ',y_val.shape)

shape of X_tra =  (287228, 200)
shape of X_val =  (31914, 200)
y_tra.shape =  (287228, 6)
y_val.shape =  (31914, 6)


In [20]:
# Read in the embedding
def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')
embedding_index = dict(get_coefs(*o.strip().split(" ")) for o in open(embedding_path, encoding="utf8"))
print('embedding stage1: read from file')

embedding stage1: read from file


In [21]:
# Assign embedding to words
word_index = tok.word_index
nb_words = min(max_features, len(word_index))
embedding_matrix = np.zeros((nb_words, embed_size))
for word, i in word_index.items():
    if i >= max_features: continue
    embedding_vector = embedding_index.get(word)
    if embedding_vector is not None: embedding_matrix[i] = embedding_vector
print('embedding stage2: embedding_vector constructed')

embedding stage2: embedding_vector constructed


In [0]:
class CyclicLR(Callback):
    """This callback implements a cyclical learning rate policy (CLR).
    The method cycles the learning rate between two boundaries with
    some constant frequency, as detailed in this paper (https://arxiv.org/abs/1506.01186).
    The amplitude of the cycle can be scaled on a per-iteration or 
    per-cycle basis.
    This class has three built-in policies, as put forth in the paper.
    "triangular":
        A basic triangular cycle w/ no amplitude scaling.
    "triangular2":
        A basic triangular cycle that scales initial amplitude by half each cycle.
    "exp_range":
        A cycle that scales initial amplitude by gamma**(cycle iterations) at each 
        cycle iteration.
    For more detail, please see paper.
    
    # Example
        ```python
            clr = CyclicLR(base_lr=0.001, max_lr=0.006,
                                step_size=2000., mode='triangular')
            model.fit(X_train, Y_train, callbacks=[clr])
        ```
    
    Class also supports custom scaling functions:
        ```python
            clr_fn = lambda x: 0.5*(1+np.sin(x*np.pi/2.))
            clr = CyclicLR(base_lr=0.001, max_lr=0.006,
                                step_size=2000., scale_fn=clr_fn,
                                scale_mode='cycle')
            model.fit(X_train, Y_train, callbacks=[clr])
        ```    
    # Arguments
        base_lr: initial learning rate which is the
            lower boundary in the cycle.
        max_lr: upper boundary in the cycle. Functionally,
            it defines the cycle amplitude (max_lr - base_lr).
            The lr at any cycle is the sum of base_lr
            and some scaling of the amplitude; therefore 
            max_lr may not actually be reached depending on
            scaling function.
        step_size: number of training iterations per
            half cycle. Authors suggest setting step_size
            2-8 x training iterations in epoch.
        mode: one of {triangular, triangular2, exp_range}.
            Default 'triangular'.
            Values correspond to policies detailed above.
            If scale_fn is not None, this argument is ignored.
        gamma: constant in 'exp_range' scaling function:
            gamma**(cycle iterations)
        scale_fn: Custom scaling policy defined by a single
            argument lambda function, where 
            0 <= scale_fn(x) <= 1 for all x >= 0.
            mode paramater is ignored 
        scale_mode: {'cycle', 'iterations'}.
            Defines whether scale_fn is evaluated on 
            cycle number or cycle iterations (training
            iterations since start of cycle). Default is 'cycle'.
    """

    def __init__(self, base_lr=0.001, max_lr=0.006, step_size=2000., mode='triangular',
                 gamma=1., scale_fn=None, scale_mode='cycle'):
        super(CyclicLR, self).__init__()

        self.base_lr = base_lr
        self.max_lr = max_lr
        self.step_size = step_size
        self.mode = mode
        self.gamma = gamma
        if scale_fn == None:
            if self.mode == 'triangular':
                self.scale_fn = lambda x: 1.
                self.scale_mode = 'cycle'
            elif self.mode == 'triangular2':
                self.scale_fn = lambda x: 1/(2.**(x-1))
                self.scale_mode = 'cycle'
            elif self.mode == 'exp_range':
                self.scale_fn = lambda x: gamma**(x)
                self.scale_mode = 'iterations'
        else:
            self.scale_fn = scale_fn
            self.scale_mode = scale_mode
        self.clr_iterations = 0.
        self.trn_iterations = 0.
        self.history = {}

        self._reset()

    def _reset(self, new_base_lr=None, new_max_lr=None,
               new_step_size=None):
        """Resets cycle iterations.
        Optional boundary/step size adjustment.
        """
        if new_base_lr != None:
            self.base_lr = new_base_lr
        if new_max_lr != None:
            self.max_lr = new_max_lr
        if new_step_size != None:
            self.step_size = new_step_size
        self.clr_iterations = 0.
        
    def clr(self):
        cycle = np.floor(1+self.clr_iterations/(2*self.step_size))
        x = np.abs(self.clr_iterations/self.step_size - 2*cycle + 1)
        if self.scale_mode == 'cycle':
            return self.base_lr + (self.max_lr-self.base_lr)*np.maximum(0, (1-x))*self.scale_fn(cycle)
        else:
            return self.base_lr + (self.max_lr-self.base_lr)*np.maximum(0, (1-x))*self.scale_fn(self.clr_iterations)
        
    def on_train_begin(self, logs={}):
        logs = logs or {}

        if self.clr_iterations == 0:
            K.set_value(self.model.optimizer.lr, self.base_lr)
        else:
            K.set_value(self.model.optimizer.lr, self.clr())        
            
    def on_batch_end(self, epoch, logs=None):
        
        logs = logs or {}
        self.trn_iterations += 1
        self.clr_iterations += 1
        K.set_value(self.model.optimizer.lr, self.clr())

        self.history.setdefault('lr', []).append(K.get_value(self.model.optimizer.lr))
        self.history.setdefault('iterations', []).append(self.trn_iterations)

        for k, v in logs.items():
            self.history.setdefault(k, []).append(v)
            

In [23]:
nlabs = 6
def get_model():
    inp = Input(shape=(maxlen, ))
    x = Embedding(max_features, embed_size, weights=[embedding_matrix], trainable=False)(inp)
    x = SpatialDropout1D(0.4)(x)
    
    # Path 1
    x1 = Bidirectional(CuDNNLSTM(64, return_sequences=True))(x)
    x1 = Conv1D(filters=64, kernel_size=3, padding='valid', activation='relu')(x1)
    
    # Path 2
    x2 = Bidirectional(CuDNNLSTM(128, return_sequences=True))(x)
    x2 = Conv1D(filters=128, kernel_size=2, padding='valid', activation='relu')(x2)
    
    avg_pool1 = GlobalAveragePooling1D()(x1)
    avg_pool2 = GlobalAveragePooling1D()(x2)
    
    max_pool1 = GlobalMaxPooling1D()(x1)
    max_pool2 = GlobalMaxPooling1D()(x2)
    
    conc = concatenate([avg_pool1, avg_pool2, max_pool1, max_pool2])
    
    #x = Bidirectional(GRU(80, dropout=0.3, recurrent_dropout=0.3, return_sequences=True))(x)
    #x = BatchNormalization()(x)
    #avg_pool = GlobalAveragePooling1D()(x)
    #max_pool = GlobalMaxPooling1D()(x)
    #conc = concatenate([avg_pool, max_pool])
    # Map each word to one of 2*2*2*2*2*2 = 64 possible classes. Since it is 
    # Bidirectional, there should be 128 possible classes
    #x = TimeDistributed(Dense(128, activation="sigmoid"))(x)
    #x = Flatten()(x)
    #x = BatchNormalization()(x)
    #x = Dropout(0.4)(x)
    outp = Dense(nlabs, activation="sigmoid")(conc)
    
    model = Model(inputs=inp, outputs=outp)
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

    return model

model = get_model()
model.summary()


Instructions for updating:
`NHWC` for data_format is deprecated, use `NWC` instead
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 200)          0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 200, 300)     45000000    input_1[0][0]                    
__________________________________________________________________________________________________
spatial_dropout1d_1 (SpatialDro (None, 200, 300)     0           embedding_1[0][0]                
__________________________________________________________________________________________________
bidirectional_1 (Bidirectional) (None, 200, 128)     187392      spatial_dropout1d_1[0][0]        
__________________________

In [24]:
file_path = "drive/jigsaw/best_model.hdf5"
RocAuc = RocAucEvaluation(validation_data=(X_val, y_val), interval=1)
save_best = ModelCheckpoint(file_path, monitor='val_loss', verbose=1, save_best_only=True, mode='auto')
early_stopping = EarlyStopping(monitor='val_loss', patience=4, mode='auto',min_delta=0.00001)
callbacks_list = [save_best, early_stopping, RocAuc]
print('callbacks defined')

callbacks defined


In [25]:
batch_size = 32
epochs = 40
hist = model.fit(X_tra, y_tra, batch_size=batch_size, epochs=epochs, validation_data=(X_val, y_val),
                 callbacks=[RocAuc, save_best, early_stopping], verbose=1)
print('model fitting complete')

Train on 287228 samples, validate on 31914 samples
Epoch 1/40
 21312/287228 [=>............................] - ETA: 20:37 - loss: 0.0749 - acc: 0.9747

287200/287228 [============================>.] - ETA: 0s - loss: 0.0493 - acc: 0.9817

287228/287228 [==============================] - 1342s 5ms/step - loss: 0.0493 - acc: 0.9817 - val_loss: 0.0427 - val_acc: 0.9832

 ROC-AUC - epoch: 1 - score: 0.989231

Epoch 00001: val_loss improved from inf to 0.04268, saving model to drive/jigsaw/best_model.hdf5
Epoch 2/40
  7008/287228 [..............................] - ETA: 21:15 - loss: 0.0440 - acc: 0.9829

287200/287228 [============================>.] - ETA: 0s - loss: 0.0420 - acc: 0.9838

287228/287228 [==============================] - 1339s 5ms/step - loss: 0.0420 - acc: 0.9838 - val_loss: 0.0383 - val_acc: 0.9851

 ROC-AUC - epoch: 2 - score: 0.989910

Epoch 00002: val_loss improved from 0.04268 to 0.03835, saving model to drive/jigsaw/best_model.hdf5
Epoch 3/40
  6944/287228 [..............................] - ETA: 21:18 - loss: 0.0426 - acc: 0.9831

...

287228/287228 [==============================] - 1334s 5ms/step - loss: 0.0352 - acc: 0.9861 - val_loss: 0.0381 - val_acc: 0.9848

 ROC-AUC - epoch: 6 - score: 0.990902

Epoch 00006: val_loss did not improve
Epoch 7/40
 10400/287228 [>.............................] - ETA: 20:41 - loss: 0.0321 - acc: 0.9874

287200/287228 [============================>.] - ETA: 0s - loss: 0.0340 - acc: 0.9865

287228/287228 [==============================] - 1339s 5ms/step - loss: 0.0340 - acc: 0.9865 - val_loss: 0.0371 - val_acc: 0.9855

 ROC-AUC - epoch: 7 - score: 0.991210

Epoch 00007: val_loss did not improve
Epoch 8/40
 10400/287228 [>.............................] - ETA: 20:44 - loss: 0.0314 - acc: 0.9873

287200/287228 [============================>.] - ETA: 0s - loss: 0.0330 - acc: 0.9870

287228/287228 [==============================] - 1338s 5ms/step - loss: 0.0330 - acc: 0.9870 - val_loss: 0.0398 - val_acc: 0.9842

 ROC-AUC - epoch: 8 - score: 0.990871

Epoch 00008: val_loss did not improve
model fitting complete


In [0]:
# Save the current model
model.save("drive/jigsaw/current_final_model.hdf5")

In [0]:
# Read in the best weights
model.load_weights(file_path)

# Make predictions with best weights
y_pred = model.predict(x_test, batch_size=1024)

In [28]:
# Save predictions
submission = pd.read_csv('drive/jigsaw/sample_submission.csv')
submission[["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]] = y_pred
submission.to_csv('drive/jigsaw/submission.csv', index=False)
print('fit and predictions complete')
submission.head()

fit and predictions complete


,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,0.991522,1.125925e-01,0.984572,2.403015e-02,0.821291,0.136614
1,0000247867823ef7,0.000365,5.863479e-07,0.000180,7.512402e-07,0.000100,0.000022
2,00013b17ad220c46,0.000701,9.839427e-06,0.000205,4.301212e-06,0.000097,0.000032
3,00017563c3f7919a,0.000143,2.287215e-08,0.000293,5.270820e-06,0.000314,0.000003
4,00017695ad8997eb,0.002581,5.791036e-06,0.000985,2.266286e-05,0.000454,0.000049
